In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf  # 주가 데이터 가져오기
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# 1️ 엔비디아 주가 데이터 가져오기
df = yf.download("NVDA", start="2024-02-15", end="2025-02-15")

# 'Close' 가격만 사용
data = df[['Close']]

# 2️ 데이터 정규화 (0~1 사이 값으로 변환)
scaler = MinMaxScaler(feature_range=(0,1))
data_scaled = scaler.fit_transform(data)

# 3️ LSTM 입력 데이터 생성 함수
def create_dataset(data, time_step=60):
    X, Y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i+time_step), 0])  # 과거 60일 데이터
        Y.append(data[i+time_step, 0])      # 다음 날 예측값
    return np.array(X), np.array(Y)

# 4️ 데이터 변환 (60일 단위 입력)
time_step = 60
X, Y = create_dataset(data_scaled, time_step)

# LSTM 입력 형태로 변환
X = X.reshape(X.shape[0], X.shape[1], 1)

# 5️ 학습/테스트 데이터 분리
split = int(len(X) * 0.8)
X_train, Y_train = X[:split], Y[:split]
X_test, Y_test = X[split:], Y[split:]

# 6️ LSTM 모델 구축
model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(time_step, 1)),
    Dropout(0.2),
    LSTM(units=50, return_sequences=False),
    Dropout(0.2),
    Dense(units=25),
    Dense(units=1)
])

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# 7️ 모델 학습
history = model.fit(X_train, Y_train, epochs=50, batch_size=16, validation_data=(X_test, Y_test), verbose=1)

# 8️ 예측 수행
predicted_price = model.predict(X_test)

# 정규화 역변환 (실제 가격으로 변환)
predicted_price = scaler.inverse_transform(predicted_price.reshape(-1, 1))
actual_price = scaler.inverse_transform(Y_test.reshape(-1, 1))

# 9️ 예측 결과 시각화
plt.figure(figsize=(10,5))
plt.plot(actual_price, label="Real Price", color="blue")
plt.plot(predicted_price, label="Predicted Price", color="red")
plt.legend()
plt.title("NVIDIA Stock Price Prediction (LSTM)")
plt.show()
